In [1]:
!pip install biopython gensim umap-learn scikit-learn==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


# Create W2V Model - Rice

3-mers / Vector: 100

In [2]:
kmer_size = 3
vector_size = 100

In [3]:
from Bio import SeqIO
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from joblib import load

def circrna_to_kmers(circrna_sequence, k=kmer_size):
  kmers = []
  for i in range(0, len(circrna_sequence)-k+1):
    kmer = circrna_sequence[i:i+k]
    kmers.append(kmer)
  return kmers

fasta_handle = open('osaj43883_genomic_seq.txt', 'r')
fasta_parser = SeqIO.parse(fasta_handle, 'fasta')

with open('oryza_corpus.txt', 'w') as corpus_handle:
  for record in fasta_parser:
    record_kmers = circrna_to_kmers(str(record.seq))
    corpus_handle.write(' '.join(record_kmers) + '\n')

w2v_model = Word2Vec(vector_size=vector_size)
w2v_model.build_vocab(corpus_file='oryza_corpus.txt')

w2v_model.corpus_count

# Training the w2v model
w2v_model.train(corpus_file='oryza_corpus.txt', total_words=w2v_model.corpus_total_words, epochs=1)

# Save the model
w2v_model.save("word2vec_model_rice_3mer_100.bin")

# Create W2V Model - Maize

3-mers / Vector: 100

In [4]:
fasta_handle = open('zma10381_genomic_seq.txt', 'r')
fasta_parser = SeqIO.parse(fasta_handle, 'fasta')

with open('maize_corpus.txt', 'w') as corpus_handle:
  for record in fasta_parser:
    record_kmers = circrna_to_kmers(str(record.seq))
    corpus_handle.write(' '.join(record_kmers) + '\n')

w2v_model = Word2Vec(vector_size=vector_size)
w2v_model.build_vocab(corpus_file='maize_corpus.txt')

w2v_model.corpus_count

# Training the w2v model
w2v_model.train(corpus_file='maize_corpus.txt', total_words=w2v_model.corpus_total_words, epochs=1)

# Save the model
w2v_model.save("word2vec_model_maize_3mer_100.bin")

# Create rice dataset W2Vec
3mer / vector: 100

In [5]:
def circrna_to_kmers(circrna_sequence, k=kmer_size):
  kmers = []
  for i in range(0, len(circrna_sequence)-k+1):
    kmer = circrna_sequence[i:i+k]
    kmers.append(kmer)
  return kmers

In [6]:
def circrna_to_vec(circrna_sequence, k=kmer_size):
  vec = np.zeros(vector_size)
  kmers = circrna_to_kmers(circrna_sequence, k=k)
  for kmer in kmers:
    if kmer in w2v_model.wv:
      vec = vec + w2v_model.wv[kmer]
  return vec

In [7]:
w2v_model = Word2Vec.load("word2vec_model_rice_3mer_100.bin")

In [8]:
circ_df = pd.read_excel('rice_db.xlsx')

In [9]:
list = circ_df['seq']

columns = [f'wc_3mer_{v+1}' for v in range(vector_size)]

df_vecs = pd.DataFrame(columns=columns)

for record in list:
  df_vecs = df_vecs._append(
      [
       dict(zip(columns, circrna_to_vec(record)))
       ],
      ignore_index=True)

df_vecs

<ipython-input-9-35e0a364540f>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vecs = df_vecs._append(


,wc_3mer_1,wc_3mer_2,wc_3mer_3,wc_3mer_4,wc_3mer_5,wc_3mer_6,wc_3mer_7,wc_3mer_8,wc_3mer_9,wc_3mer_10,...,wc_3mer_91,wc_3mer_92,wc_3mer_93,wc_3mer_94,wc_3mer_95,wc_3mer_96,wc_3mer_97,wc_3mer_98,wc_3mer_99,wc_3mer_100
0,-0.114550,8.709467,-24.012228,-8.613446,18.093548,9.263698,-1.620138,-13.933384,10.277954,5.983017,...,-3.909451,-7.255758,-4.081003,7.047246,-3.436967,-10.292043,6.197461,-6.063741,-10.206034,-3.089770
1,-65.959607,44.943739,-127.204847,-49.205670,21.851977,4.422079,-61.379360,-14.825413,-55.541652,33.088241,...,15.349517,-5.572336,-21.901505,15.447046,15.465014,15.264171,21.415839,21.776205,-88.552441,-15.640409
2,41.406683,-0.112979,10.573092,-15.862843,69.725090,62.108435,13.985081,-5.202757,41.996200,40.917527,...,9.115822,0.574996,-27.427537,26.609707,11.901475,-54.170248,-32.457364,-15.680423,-0.892006,-27.017880
3,-26.714316,-14.492211,-34.893208,-48.425272,4.287348,23.565343,-19.946376,5.547091,-60.870550,22.216801,...,23.287733,-8.624785,-19.587672,9.516696,14.729817,2.090331,6.735275,42.251084,-47.116036,-21.105272
4,27.174758,8.573330,2.691037,28.440111,38.831811,10.781812,27.501626,-16.137740,59.954095,8.407088,...,-21.630480,-11.773856,-16.887803,32.656249,1.545665,-31.344329,-37.487578,0.327636,21.849911,-13.385421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63043,-35.514152,-20.887956,-29.103053,-14.741384,-26.303455,-14.532384,3.109194,9.729682,-45.892952,-17.552039,...,-4.845106,-25.643219,-14.525121,-1.496573,22.123897,23.395100,10.386098,27.494896,-22.710221,-14.643069
63044,-4.627548,-75.596588,66.126815,23.226359,-23.814145,21.092233,77.733558,60.251850,10.671653,-95.669383,...,-33.732328,-34.740534,-10.689965,55.858803,94.575089,8.050199,-61.650838,66.855732,68.859820,-98.389106
63045,7.846368,-37.117494,51.700139,13.442700,-0.999745,39.494642,52.497463,47.431460,18.804811,-52.944599,...,-17.004266,-5.049049,-1.975543,50.706912,71.063941,-21.738402,-69.801899,48.826817,60.348895,-78.580770
63046,-4.441904,2.869366,-8.800975,-23.783700,12.840042,-0.160727,-13.484560,-9.966495,3.851810,-7.974538,...,-2.729394,-1.169394,-2.621319,16.252393,10.420365,12.527998,-9.512905,22.593617,-33.524130,-15.119008


In [10]:
new_df = pd.concat([circ_df, df_vecs], axis=1)

new_df.head()

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_3mer_91,wc_3mer_92,wc_3mer_93,wc_3mer_94,wc_3mer_95,wc_3mer_96,wc_3mer_97,wc_3mer_98,wc_3mer_99,wc_3mer_100
0,osa-circ1-OS01T0723400,1:30167620-30167771_+,OS01T0723400,OS01T0723400-01,-,multipleTissue,1,30167620,30167771,+,...,-3.909451,-7.255758,-4.081003,7.047246,-3.436967,-10.292043,6.197461,-6.063741,-10.206034,-3.089770
1,osa-circ2-OS03T0223400,3:6461672-6462146_-,OS03T0223400,OS03T0223400-01,-,multipleTissue,3,6461672,6462146,-,...,15.349517,-5.572336,-21.901505,15.447046,15.465014,15.264171,21.415839,21.776205,-88.552441,-15.640409
2,osa-circ3-OS11T0210300,11:5715883-5716030_-,OS11T0210300,OS11T0210300-02,-,multipleTissue,11,5715883,5716030,-,...,9.115822,0.574996,-27.427537,26.609707,11.901475,-54.170248,-32.457364,-15.680423,-0.892006,-27.017880
3,osa-circ4-OS02T0200900,2:5631023-5631244_-,OS02T0200900,OS02T0200900-02,-,multipleTissue,2,5631023,5631244,-,...,23.287733,-8.624785,-19.587672,9.516696,14.729817,2.090331,6.735275,42.251084,-47.116036,-21.105272
4,osa-circ5-OS05T0494800,5:24302336-24302448_+,OS05T0494800,OS05T0494800-01,-,multipleTissue,5,24302336,24302448,+,...,-21.630480,-11.773856,-16.887803,32.656249,1.545665,-31.344329,-37.487578,0.327636,21.849911,-13.385421


In [11]:
new_df.to_excel('rice_w2vec_3mer_100_dataset.xlsx', index=False)

# Create maize dataset W2Vec
3mer / vector: 100

In [12]:
w2v_model = Word2Vec.load("word2vec_model_maize_3mer_100.bin")

In [13]:
circ_df = pd.read_excel('maize_db.xlsx')

In [14]:
list = circ_df['seq']

columns = [f'wc_3mer_{v+1}' for v in range(vector_size)]

df_vecs = pd.DataFrame(columns=columns)

for record in list:
  df_vecs = df_vecs._append(
      [
       dict(zip(columns, circrna_to_vec(record)))
       ],
      ignore_index=True)

df_vecs

<ipython-input-14-35e0a364540f>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vecs = df_vecs._append(


,wc_3mer_1,wc_3mer_2,wc_3mer_3,wc_3mer_4,wc_3mer_5,wc_3mer_6,wc_3mer_7,wc_3mer_8,wc_3mer_9,wc_3mer_10,...,wc_3mer_91,wc_3mer_92,wc_3mer_93,wc_3mer_94,wc_3mer_95,wc_3mer_96,wc_3mer_97,wc_3mer_98,wc_3mer_99,wc_3mer_100
0,-12.352000,-8.186747,-32.662804,16.487791,-23.655421,-0.925946,-11.655487,-13.766879,11.732281,4.410189,...,-4.353594,7.649934,26.268557,-19.307365,-7.781124,3.164306,20.016416,16.676487,46.823639,19.710762
1,-154.249942,13.478397,-76.532436,104.256894,10.825853,-66.174551,260.125945,80.300060,105.984867,151.393883,...,-96.277733,-135.998009,106.478000,-60.739613,121.448168,-9.148820,246.986351,-36.946298,174.563397,27.953961
2,37.059497,-4.938679,-28.057114,6.435967,-26.656563,-5.318831,-12.411551,-34.743334,-12.284319,-23.437436,...,-7.727312,23.650509,-5.016291,-18.919755,-36.442056,5.596979,16.153520,44.446752,-6.045732,-5.582788
3,-12.484866,-14.541434,-19.247971,13.559102,-20.394149,-0.404936,-7.857663,-16.614924,13.330618,6.714429,...,-4.285113,13.156789,10.370599,6.732130,-0.930136,7.914002,3.937232,8.497680,35.805687,15.247144
4,-41.879316,15.727781,-36.669743,37.235389,-5.904601,38.810254,33.935180,-28.980305,50.900249,40.263193,...,-13.225035,-44.428604,11.982961,-34.749385,36.678418,-11.976684,58.400786,-0.792568,31.533359,19.249885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38780,71.164411,-1.239503,-14.628161,-1.196922,-29.734705,1.395032,-41.462242,-42.850887,-13.437058,-46.724233,...,-3.139219,36.246606,-25.837743,-24.121229,-55.724027,-3.449665,-7.282602,49.043358,-37.936987,-12.755764
38781,7.323010,9.828890,-18.114589,-11.624765,5.167649,9.145715,7.651934,-21.106976,-1.634700,-2.538340,...,-2.748252,-8.968890,-15.035174,-16.033198,-0.424244,-6.335166,11.615390,17.138981,-6.214842,-9.366027
38782,102.417760,15.276899,-8.140058,-13.256023,-18.616275,34.847627,-82.458079,-84.508404,25.397271,-68.654236,...,2.480052,45.579914,-58.230658,-15.391135,-103.363007,-26.675366,-34.550304,40.673612,-75.494079,-4.127109
38783,100.349026,31.098426,-65.883432,-22.878046,-4.542618,-97.736578,132.416143,-10.144127,-90.447500,-77.562795,...,-21.850815,9.442444,-12.625689,-91.238807,-0.156580,37.112311,109.579157,75.244680,-22.207254,-82.014492


In [15]:
new_df = pd.concat([circ_df, df_vecs], axis=1)

new_df.head()

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_3mer_91,wc_3mer_92,wc_3mer_93,wc_3mer_94,wc_3mer_95,wc_3mer_96,wc_3mer_97,wc_3mer_98,wc_3mer_99,wc_3mer_100
0,zma-circ1-Zm00001d002325,2:10317309-10317467_-,Zm00001d002325,Zm00001d002325_T001,-,multipleTissue,2,10317309,10317467,-,...,-4.353594,7.649934,26.268557,-19.307365,-7.781124,3.164306,20.016416,16.676487,46.823639,19.710762
1,zma-circ2-Zm00001d038675,6:162376852-162378246_+,Zm00001d038675,Zm00001d038675_T004,-,multipleTissue,6,162376852,162378246,+,...,-96.277733,-135.998009,106.478000,-60.739613,121.448168,-9.148820,246.986351,-36.946298,174.563397,27.953961
2,zma-circ3-Zm00001d038163,6:150032431-150032595_+,Zm00001d038163,Zm00001d038163_T001,-,multipleTissue,6,150032431,150032595,+,...,-7.727312,23.650509,-5.016291,-18.919755,-36.442056,5.596979,16.153520,44.446752,-6.045732,-5.582788
3,zma-circ4-Zm00001d049552,4:34381638-34381747_-,Zm00001d049552,Zm00001d049552_T004,-,multipleTissue,4,34381638,34381747,-,...,-4.285113,13.156789,10.370599,6.732130,-0.930136,7.914002,3.937232,8.497680,35.805687,15.247144
4,zma-circ5-Zm00001d032567,1:230724608-230725226_-,Zm00001d032567,Zm00001d032567_T001,-,multipleTissue,1,230724608,230725226,-,...,-13.225035,-44.428604,11.982961,-34.749385,36.678418,-11.976684,58.400786,-0.792568,31.533359,19.249885


In [17]:
new_df.to_excel('maize_w2vec_3mer_100_dataset.xlsx', index=False)